useful links:

- Data Preparation for Variable Length Input Sequences, URL: https://machinelearningmastery.com/data-preparation-variable-length-input-sequences-sequence-prediction/
- Masking and padding with Keras, URL: https://www.tensorflow.org/guide/keras/masking_and_padding
- Step-by-step understanding LSTM Autoencoder layers, URL: https://towardsdatascience.com/step-by-step-understanding-lstm-autoencoder-layers-ffab055b6352XX, 
- Understanding input_shape parameter in LSTM with Keras, URL: https://stats.stackexchange.com/questions/274478/understanding-input-shape-parameter-in-lstm-with-keras
- tf.convert_to_tensor, URL: https://www.tensorflow.org/api_docs/python/tf/convert_to_tensor
- ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int) in Python, URL: https://datascience.stackexchange.com/questions/82440/valueerror-failed-to-convert-a-numpy-array-to-a-tensor-unsupported-object-type
- How to Identify and Diagnose GAN Failure Modes, URL: https://machinelearningmastery.com/practical-guide-to-gan-failure-modes/
- How to Develop a GAN for Generating MNIST Handwritten Digits
, URL: https://machinelearningmastery.com/how-to-develop-a-generative-adversarial-network-for-an-mnist-handwritten-digits-from-scratch-in-keras/
- How to Visualize a Deep Learning Neural Network Model in Keras
, URL: https://machinelearningmastery.com/visualize-deep-learning-neural-network-model-keras/
- How to Implement GAN Hacks in Keras to Train Stable Models
, URL: https://machinelearningmastery.com/how-to-code-generative-adversarial-network-hacks/
- Tips for Training Stable Generative Adversarial Networks
, URL: https://machinelearningmastery.com/how-to-train-stable-generative-adversarial-networks/
- How to Implement GAN Hacks in Keras to Train Stable Models
, URL: https://machinelearningmastery.com/how-to-code-generative-adversarial-network-hacks/
- How to Configure Image Data Augmentation in Keras, URL: https://machinelearningmastery.com/how-to-configure-image-data-augmentation-when-training-deep-learning-neural-networks/


In [139]:
"""
* Copyright 2020, Maestria de Humanidades Digitales,
* Universidad de Los Andes
*
* Developed for the Msc graduation project in Digital Humanities
*
* This program is free software: you can redistribute it and/or modify
* it under the terms of the GNU General Public License as published by
* the Free Software Foundation, either version 3 of the License, or
* (at your option) any later version.
*
* This program is distributed in the hope that it will be useful,
* but WITHOUT ANY WARRANTY; without even the implied warranty of
* MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
* GNU General Public License for more details.
*
* You should have received a copy of the GNU General Public License
* along with this program.  If not, see <http://www.gnu.org/licenses/>.
"""

# ===============================
# native python libraries
# ===============================
import re
import random
import math
import json
import csv
import cv2
import datetime
import copy
import gc
from statistics import mean
from collections import OrderedDict
from collections import Counter
from collections import deque

# ===============================
# extension python libraries
# ===============================
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import matplotlib.pyplot as plt

# natural language processing packages
import gensim
from gensim import models
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# downloading nlkt data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# sample handling sklearn package
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import make_scorer
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain

# # Keras + Tensorflow ML libraries
import tensorflow as tf
# from tensorflow.keras.layers

# preprocessing and processing
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping

# models
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model

# shapping layers
from tensorflow.keras.layers import Masking
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Concatenate

# basic layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import TimeDistributed

# data processing layers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import SpatialDropout1D

# recurrent and convolutional layers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import GlobalMaxPool2D
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import ZeroPadding2D

# activarion function
from tensorflow.keras.layers import LeakyReLU

# optimization loss functions
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.optimizers import SGD # OJO!
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam # OJO!
from tensorflow.keras.optimizers import Adadelta # OJO!
from tensorflow.keras.optimizers import Adagrad # OJO!

# image augmentation and processing
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator

# ===============================
# developed python libraries
# ===============================

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [140]:
# GPU config if I have
physical_devices = tf.config.list_physical_devices("GPU")
print(physical_devices)

[]


# FUNCTION DEFINITION

In [141]:
'''
A UDF to convert input data into 3-D
array as required for LSTM network.

taken from https://towardsdatascience.com/step-by-step-understanding-lstm-autoencoder-layers-ffab055b6352
'''
def temporalize(data, lookback):
    output_X = list()
    for i in range(len(X)-lookback-1):
        temp = list()
        for j in range(1,lookback+1):
            # Gather past records upto the lookback period
            temp.append(data[[(i+j+1)], :])
        temp = np.array(temp, dtype="object")
        output_X.append(temp)
    output_X = np.array(output_X, dtype="object")
    return output_X

In [142]:
# function to read the image from file with cv2
def read_img(img_fpn):
    ans = cv2.imread(img_fpn, cv2.IMREAD_UNCHANGED)
    return ans

In [143]:
# fuction to scale the image and reduce cv2
def scale_img(img, scale_pct):

    width = int(img.shape[1]*scale_pct/100)
    height = int(img.shape[0]*scale_pct/100)
    dim = (width, height)
    # resize image
    ans = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    return ans

In [144]:
# function to standarize image, has 2 types, from 0 to 1 and from -1 to 1
def std_img(img, minv, maxv, stype="std"):
    ans = None
    rangev = maxv - minv

    if stype == "std":
        ans = img.astype("float32")/float(rangev)
    
    elif stype == "ctr":
        rangev = float(rangev/2)
        ans = (img.astype("float32")-rangev)/rangev
    # ans = pd.Series(ans)
    return ans

In [145]:
# function to pad the image in the center
def pad_img(img, h, w, img_type):
    #  in case when you have odd number
    ans = None
    top_pad = np.floor((h - img.shape[0]) / 2).astype(np.uint8) # floor
    bottom_pad = np.ceil((h - img.shape[0]) / 2).astype(np.uint8)
    right_pad = np.ceil((w - img.shape[1]) / 2).astype(np.uint8)
    left_pad = np.floor((w - img.shape[1]) / 2).astype(np.uint8) # floor
    # print((top_pad, bottom_pad), (left_pad, right_pad))
    if img_type == "rgb":
        ans = np.copy(np.pad(img, ((top_pad, bottom_pad), (left_pad, right_pad), (0, 0)), mode="constant", constant_values=0.0))   
    if img_type == "bw":
        ans = np.copy(np.pad(img, ((int(top_pad), int(bottom_pad)), (int(left_pad), int(right_pad))), mode="constant", constant_values=0))

    return ans

In [146]:
def update_shape(src_df, img_col, shape_col):

    ans = src_df
    src_col = list(ans[img_col])
    tgt_col = list()

    # ansdict = {}
    for data in src_col:
        tshape = data.shape
        tgt_col.append(tshape)

    ans[shape_col] = tgt_col
    return ans

In [147]:
# function to padd the images in the dataset, needs the shape, the type of image and the src + tgt columns of the frame to work with
def padding_images(src_df, src_col, tgt_col, max_shape, img_type):
    # ans = src_df
    src_images = src_df[src_col]
    tgt_images = list()
    max_x, max_y = max_shape[0], max_shape[1]

    for timg in src_images:        
        pimg = pad_img(timg, max_y, max_x, img_type)
        tgt_images.append(pimg)

    src_df[tgt_col] = tgt_images
    return src_df

In [148]:
# function to load the images in in memory
def get_images(rootf, src_df, src_col, tgt_col, scale_pct):
    ans = src_df
    src_files = list(ans[src_col])
    tgt_files = list()

    # ansdict = {}
    for tfile in src_files:
        tfpn = os.path.join(rootf, tfile)
        timg = read_img(tfpn)
        timg = scale_img(timg, scale_pct)
        tgt_files.append(timg)

    ans[tgt_col] = tgt_files
    return ans

In [149]:
# function to augment the images in the dataset and virtualy exapnd the training examples
def augment_images(src_df, src_col, tgt_col, syth_num):

    cols = [list(src_df.columns.values)]
    # print(cols)
    ans = pd.DataFrame()
    other_cols = list(src_df.columns.values)
    other_cols.remove(tgt_col)
    other_cols.remove(src_col)
    # print(other_cols)

    for index, row in src_df.iterrows():
        t_txt = row[src_col]
        t_img = row[tgt_col]
        t_tags = row[other_cols]

        gen_rows = list()
        for i in range(syth_num):

            gen_tags = copy.deepcopy(t_tags)
            gen_img = syth_img(t_img)
            gen_txt = syth_text(t_txt)
            # print(type(gen_tags), type(gen_img)) 
            gen_tags[tgt_col] = gen_img
            gen_tags[src_col] = gen_txt
            gen_rows.append(gen_tags)
            # print(gen_tags) # , type(gen_img)) 
            # [other_cols], row[tgt_col])
        
        ans = ans.append(gen_rows, ignore_index=True)

    return ans

In [150]:
# function to standarize the images in the dataset, it has 2 options
def standarize_images(src_df, src_col, tgt_col, img_type, std_opt):
    src_images = src_df[src_col]
    tgt_images = list()

    for timg in src_images:
        # pcolor image
        if img_type == "rgb":
            timg = np.asarray(timg, dtype="object")
        
        # b&w image
        if img_type == "rb":
            timg = np.asarray(timg) #, dtype="uint8")
            timg = timg[:,:,np.newaxis]
            timg = np.asarray(timg, dtype="object")
        
        # std_opt affect the standarization results
        # result 0.0 < std_timg < 1.0
        # result -1.0 < std_timg < 1.0
        std_timg = std_img(timg, 0, 255, std_opt)
        tgt_images.append(std_timg)

    src_df[tgt_col] = tgt_images
    return src_df

In [151]:
# function than rotates the original image to create a new example
def syth_rgb_img(data):

    samples = expand_dims(data, 0)
    datagen = ImageDataGenerator(rotation_range=90)
    ans = datagen.flow(samples, batch_size=1)
    ans = ans[0].astype("uint8")
    ans = np.squeeze(ans, 0)
    return ans

In [152]:
# function to get the max shape in the image dataset
def get_mshape(shape_data, imgt):

    max_x, max_y, max_ch = 0, 0, 0
    shape_data = list(shape_data)
    ans = None

    if imgt == "rgb":

        for tshape in shape_data:
            # tshape = eval(tshape)
            tx, ty, tch = tshape[0], tshape[1], tshape[2]

            if tx > max_x:
                max_x = tx
            if ty > max_y:
                max_y = ty
            if tch > max_ch:
                max_ch = tch
            
        ans = (max_x, max_y, max_ch)
    
    elif imgt == "bw":

        for tshape in shape_data:
            # tshape = eval(tshape)
            tx, ty = tshape[0], tshape[1]

            if tx > max_x:
                max_x = tx
            if ty > max_y:
                max_y = ty
            
        ans = (max_x, max_y)
        
    return ans

In [153]:
'''
A UDF to convert input data into 3-D
array as required for LSTM network.

taken from https://towardsdatascience.com/step-by-step-understanding-lstm-autoencoder-layers-ffab055b6352
'''
def temporalize(data, lookback):
    output_X = list()
    for i in range(len(data)-lookback-1):
        temp = list()
        for j in range(1,lookback+1):
            # Gather past records upto the lookback period
            temp.append(data[[(i+j+1)], :])
        temp = np.array(temp, dtype="object")
        output_X.append(temp)
    output_X = np.array(output_X, dtype="object")
    return output_X

In [154]:
# format the pandas df data into usable word dense vector representation, YOU NEED IT FOR THE CSV to be useful!
def format_dvector(work_corpus):

    ans = list()
    for dvector in work_corpus:
        dvector = eval(dvector)
        dvector = np.asarray(dvector)
        ans.append(dvector)
    ans = np.asarray(ans, dtype="object")
    return ans

In [155]:
# funct to concatenate all label columns into one for a single y in ML training, returns a list
def concat_labels(row, cname):

    ans = list()
    for c in cname:
        r = row[c]
        r = eval(r)
        ans = ans + r

    return ans

In [156]:
# function to save the ML model
def save_model(model, m_path, m_file):

    fpn = os.path.join(m_path, m_file)
    fpn = fpn + ".h5"
    # print(fpn)
    model.save(fpn)

In [157]:
# function to load the ML model
def load_model(m_path, m_file):

    fpn = os.path.join(m_path, m_file)
    fpn = fpn + ".h5"
    model = keras.models.load_model(fpn)
    return model

In [158]:
# function to cast dataframe and avoid problems with keras
def cast_batch(X_txt, X_img, y, labels):
    X_txt = np.asarray(X_txt).astype("float32")
    X_img = np.asarray(X_img).astype("float32")
    y = np.asarray(y).astype("float32")
    labels = np.asarray(labels).astype("float32")
    return X_txt, X_img, y, labels

In [159]:
# function to select real elements to train the discriminator
def gen_real_samples(X_txt, X_img, y, y_labels, sample_size, half_batch):

    rand_index = np.random.randint(0, sample_size, size=half_batch)
    Xt_real = X_txt[rand_index]
    Xi_real = X_img[rand_index]
    y_real = y[rand_index]
    yl_real = y_labels[rand_index]
    # noise = np.random.uniform(0.0, 0.05, size=y_real.shape)
    # y_real = np.subtract(y_real, noise)
    Xt_real, Xi_real, y_real, yl_real = cast_batch(Xt_real, Xi_real, y_real, yl_real)
    real_data = (Xt_real, Xi_real, y_real, yl_real)
    return real_data

In [160]:
# function to create fake elements to train the discriminator
def gen_fake_samples(gen_model, txt_size, cat_size, label_size, half_batch):
    
    # random text
    Xt_fake = gen_latent_txt(txt_size, half_batch)
    # random labels for the text
    yl_fake = get_fake_neglbls(label_size, half_batch)
    # random generated image from the model
    Xi_fake = gen_model.predict([Xt_fake, yl_fake])
    # marking the images as fake in all accounts
    y_fake = get_fake_negative(cat_size, half_batch)

    # casting data type
    Xt_fake, Xi_fake, y_fake, yl_fake = cast_batch(Xt_fake, Xi_fake, y_fake, yl_fake)
    fake_data = (Xt_fake, Xi_fake, y_fake, yl_fake)
    
    return fake_data

In [161]:
# function to create one fake + real samples to train the discriminator
def complete_batch(Xt_real, Xi_real, y_real, Xt_fake, Xi_fake, y_fake):

    # this batch needs txt to create images, the images themselves, and the images labels
    Xt = np.concatenate((Xt_real, Xt_fake), axis=0)
    Xi = np.concatenate((Xi_real, Xi_fake), axis=0)
    y = np.concatenate((y_real, y_fake), axis=0)
    # y = np.concatenate((y_real, y_fake), axis=0)
    # Xt, Xi, y = cast_batch(Xt, Xi, y)
    
    return Xt, Xi, y

In [162]:
# function to generate random/latent text for image generator
def gen_latent_txt(txt_shape, n_samples):

    ans = None
    for i in range(n_samples):

        # be aware of this!!!!!!!
        noise = np.random.normal(0.0, 1.0, size=txt_shape)
        if ans is None:
            txt = np.expand_dims(noise, axis=0)
            ans = txt
        else:
            img = np.expand_dims(txt, axis=0)
            ans = np.concatenate((ans, txt), axis=0)
    # print(ans.shape)
    # print(ans[0])
    return ans

In [163]:
# tfunction to smooth the fake positives
def smooth_positive_labels(y):
	return y - 0.3 + (np.random.random(y.shape)*0.5)

In [164]:
# tfunction to smooth the fake negatives
def smooth_negative_labels(y):
	return y + np.random.random(y.shape)*0.3

In [165]:
# randomly flip some labels
def noisy_labels(y, p_flip):
	# determine the number of labels to flip
	n_select = int(p_flip * y.shape[0])
	# choose labels to flip
	flip_ix = np.random.choice([i for i in range(y.shape[0])], size=n_select)
	# invert the labels in place
	y[flip_ix] = 1 - y[flip_ix]
	return y

In [166]:
# generate fake true categories for the generator
def get_fake_positive(cat_size, batch_size):

    sz = (batch_size, cat_size)
    ans = np.ones(sz)
    # smooothing fakes
    ans = smooth_positive_labels(ans)
    ans = ans.astype("float32")
    # ans = np.ones((batch_size, cat_size), dtype="float32")
    return ans

In [167]:
# generate fake negative category to train the GAN
def get_fake_negative(cat_size, batch_size):

    sz = (batch_size, cat_size)
    ans = np.zeros(sz)
    ans = smooth_negative_labels(ans)
    ans = ans.astype("float32")
    # ans = np.ones((batch_size, cat_size), dtype="float32")
    return ans

In [168]:
# function to generate fake labels to train the gan
def get_fake_neglbls(label_size, batch_size):

    sz = (batch_size, label_size)
    # ans = np.random.randint(0,1, size=sz)
    ans = np.zeros(sz)
    ans = smooth_negative_labels(ans)
    # ans = smooth_positive_labels(ans)
    ans = ans.astype("float32")
    # ans = np.ones((batch_size, cat_size), dtype="float32")
    return ans

In [169]:
# get_fake_labels
# function to generate fake labels to train the gan
def get_fake_poslbls(label_size, batch_size):

    sz = (batch_size, label_size)
    # ans = np.random.randint(0,1, size=sz)
    ans = np.ones(sz)
    # ans = smooth_negative_labels(ans)
    ans = smooth_positive_labels(ans)
    ans = ans.astype("float32")
    # ans = np.ones((batch_size, cat_size), dtype="float32")
    return ans

In [170]:
# generate an expanded bath of images for training with some synthetic ones
def gen_synthetic_images(X_img, img_size, batch_size, synth_size):

    ans = None

    # iterating the images and synth new ones
    for img in X_img:
        gen_img = None

        # creating new ones
        for j in range(synth_size):

            if gen_img is None:
                timg = syth_std_img(img)
                timg = np.expand_dims(timg, axis=0)
                gen_img = timg
            
            else:
                timg = syth_std_img(img)
                timg = np.expand_dims(timg, axis=0)
                gen_img = np.concatenate((gen_img, timg), axis=0)
        
        # adding it to the training batch
        if ans is None:
            ans = gen_img

        else:
            ans = np.concatenate((ans, gen_img), axis=0)

    return ans

In [171]:
# function to create text similar to the original one with 5% of noise
def syth_text(data, nptc=0.02):

    ans = None
    noise = np.random.normal(0, nptc, data.shape)
    ans = data + noise
    return ans

In [172]:
# synthetizing a noisy std image from real data
def syth_std_img(data):

    samples = np.expand_dims(data, 0)
    datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=15)
    # datagen = ImageDataGenerator(rotation_range=10, horizontal_flip=True, vertical_flip=True)
    ans = datagen.flow(samples, batch_size=1)
    ans = ans[0].astype("float32")
    ans = np.squeeze(ans, 0)
    return ans

In [173]:
# function to create new categories with some noise, default 5%
def syth_categories(data, nptc=0.03):

    ans = None
    noise = np.random.normal(0, nptc, data.shape)
    ans = data + noise
    return ans

In [174]:
# function to artificially span a batch with some noise and alterations by an specific number
def expand_samples(data, synth_batch):

    X_txt = data[0]
    X_img = data[1]
    y = data[2]
    labels = data[3]

    # creating the exapnded batch response
    Xe_txt, Xe_img, ye, lbe = None, None, None, None

    # iterating in the original batch
    for Xtt, Xit, yt, lb in zip(X_txt, X_img, y, labels):

        # temporal synth minibatch per original image
        synth_Xt, synth_Xi, synth_y, synth_lb = None, None, None, None

        # synthetizing artificial data for the batch
        for i in range(synth_batch):

            # generating first element
            if (synth_Xt is None) and (synth_Xi is None) and (synth_y is None) and (synth_lb is None):
                # gen text
                gen_Xt = copy.deepcopy(Xtt)
                gen_Xt = np.array(gen_Xt)
                gen_Xt = np.expand_dims(gen_Xt, axis=0)
                synth_Xt = gen_Xt

                # gen images
                gen_Xi = syth_std_img(Xit)
                gen_Xi = np.expand_dims(gen_Xi, axis=0)
                synth_Xi = gen_Xi

                # gen category
                gen_yt = syth_categories(yt)
                gen_yt = np.expand_dims(gen_yt, axis=0)
                synth_y = gen_yt

                # gen labels
                gen_lb = syth_categories(lb)
                gen_lb = np.expand_dims(gen_lb, axis=0)
                synth_lb = gen_lb

            # generatin the rest of the elements
            else:
                # gen text
                gen_Xt = syth_text(Xtt)
                gen_Xt = np.expand_dims(gen_Xt, axis=0)
                synth_Xt = np.concatenate((synth_Xt, gen_Xt), axis=0)

                # gen images
                gen_Xi = syth_std_img(Xit)
                gen_Xi = np.expand_dims(gen_Xi, axis=0)
                synth_Xi = np.concatenate((synth_Xi, gen_Xi), axis=0)

                # gen category
                gen_yt = syth_categories(yt)
                gen_yt = np.expand_dims(gen_yt, axis=0)
                synth_y = np.concatenate((synth_y, gen_yt), axis=0)
        
                # gen labels
                gen_lb = syth_categories(lb)
                gen_lb = np.expand_dims(gen_lb, axis=0)
                synth_lb = np.concatenate((synth_lb, gen_lb), axis=0)

        # adding the first part to the training batch
        if (Xe_txt is None) and (Xe_img is None) and (ye is None) and (lbe is None):
            # adding text
            Xe_txt = synth_Xt
            # adding images
            Xe_img = synth_Xi
            # adding categories
            ye = synth_y
            # adding labels
            lbe = synth_lb

        # adding the rest of the batch
        else:
            # adding text
            Xe_txt = np.concatenate((Xe_txt, synth_Xt), axis=0)
            # adding images
            Xe_img = np.concatenate((Xe_img, synth_Xi), axis=0)
            # adding category
            ye = np.concatenate((ye, synth_y), axis=0)
            # adding labels
            lbe = np.concatenate((lbe, synth_lb), axis=0)

    Xe_txt, Xe_img, ye, lbe = cast_batch(Xe_txt, Xe_img, ye, lbe)

    e_data = (Xe_txt, Xe_img, ye, lbe)

    return e_data

In [175]:
# def drift_labels(Xt_real, Xi_real, y_real, Xt_fake, Xi_fake, y_fake, batch_size, drift_pct):
def drift_labels(real_data, fake_data, batch_size, drift_pct):

    # real data asignation
    Xt_real = real_data[0]
    Xi_real = real_data[1]
    y_real = real_data[2]
    yl_real = real_data[3]

    # fake data asignation
    Xt_fake = fake_data[0]
    Xi_fake = fake_data[1]
    y_fake = fake_data[2]
    yl_fake = fake_data[3]

    # setting the size for the drift labels
    drift_size = int(math.ceil(drift_pct*batch_size))
    # random index for drift elements!!!
    rand_drifters = np.random.choice(batch_size, size=drift_size, replace=False)
    # print("batch size", batch_size, "\nrandom choise to change", drift_size, "\n", rand_drifters)

    for drift in rand_drifters:

        # copying temporal real data
        Xt_drift = copy.deepcopy(Xt_real[drift])
        Xi_drift = copy.deepcopy(Xi_real[drift])
        y_drift = copy.deepcopy(y_real[drift])
        yl_drift = copy.deepcopy(yl_real[drift])
        # print("OG real y:", y_drift)
        # print("OG fake y:", y_fake[drift])
        
        # replacing real with fakes
        Xt_real[drift] = copy.deepcopy(Xt_fake[drift])
        Xi_real[drift] = copy.deepcopy(Xi_fake[drift])
        y_real[drift] = copy.deepcopy(y_fake[drift])
        yl_real[drift] = copy.deepcopy(yl_fake[drift])
        # print("New real y:", y_real[drift])

        # updating fakes with temporal original
        Xt_fake[drift] = Xt_drift
        Xi_fake[drift] = Xi_drift
        y_fake[drift] = y_drift
        yl_fake[drift] = yl_drift
        # print("New fake y:", y_fake[drift])

        real_data = (Xt_real, Xi_real, y_real, yl_real)
        fake_data = (Xt_fake, Xi_fake, y_fake, yl_fake)

    return real_data, fake_data

In [176]:
# functioon to log the training results
def test_model(epoch, gen_model, dis_model, X_txt, X_img, y, labels, half_batch, report_fn_path, synth_batch):

    # register shape
    txt_shape = X_txt[0].shape
    img_shape = X_img[0].shape
    cat_shape = y[0].shape
    label_shape = labels[0].shape

    # sample size
    txt_size = X_txt.shape[0]
    img_size = X_img.shape[0]
    cat_size = y.shape[0]
    label_size = labels.shape[0]

    # select real txt2img for discrimintator
    real_data = gen_real_samples(X_txt, X_img, y, labels, img_size, half_batch)
    # expand the training sample for the discriminator
    # real_data = expand_samples(real_data, synth_batch)
    # print(Xt_real.shape, Xi_real.shape, y_real.shape, yl_real.shape)

    # create false txt for txt2img for generator
    fake_data = gen_fake_samples(gen_model, txt_shape, cat_shape[0], label_shape[0], half_batch)
    # expand the training sample for the discriminator
    # fake_data = expand_samples(fake_data, synth_batch)
    # print(Xt_fake.shape, Xi_fake.shape, y_fake.shape, yl_fake.shape)

    # fake data asignation
    Xi_fake = fake_data[1]

    # plotting gen images
    plot_gen_images(Xi_fake, epoch, report_fn_path, 3)

    # real_batch = int((half_batch*synth_batch)/2)
    real_batch = int(half_batch)

    # drift labels to confuse the model
    real_data, fake_data = drift_labels(real_data, fake_data, half_batch, 0.05)

    # real data asignation
    Xt_real = real_data[0]
    Xi_real = real_data[1]
    y_real = real_data[2]
    yl_real = real_data[3]

    # fake data asignation
    Xt_fake = fake_data[0]
    Xi_fake = fake_data[1]
    y_fake = fake_data[2]
    yl_fake = fake_data[3]

    # evaluate model
    test_real = dis_model.evaluate([Xi_real, yl_real], y_real, verbose=0)
    test_fake = dis_model.evaluate([Xi_fake, yl_fake], y_fake, verbose=0)
    # test_cgen = gen_model.evaluate([Xt_fake, yl_fake], y_fake, verbose=0)

    # summarize discriminator performance
    print("Batch Size %d -> Samples: Fake: %d & Real: %d" % (half_batch*synth_batch, real_batch, real_batch))
    print(">>> Test Fake -> Acc: %.3f || Loss: %.3f" % (test_fake[1], test_fake[0]))
    print(">>> Test Real -> Acc: %.3f || Loss: %.3f" % (test_real[1], test_real[0]))
    # print(">>> Test Gen -> Acc: %.3f || Loss: %.3f" % (test_cgen[1], test_cgen[0]))

In [177]:
# function to standarize image, has 2 types, from 0 to 1 and from -1 to 1
def inv_std_img(img, minv, maxv, stype="std"):
    ans = None
    rangev = maxv - minv

    if stype == "std":
        ans = img*rangev
        ans = np.asarray(ans).astype("uint8")

    elif stype == "ctr":
        rangev = float(rangev/2)
        ans = img+rangev
        ans = ans*rangev
        ans = np.asarray(ans).astype("uint8")

    return ans

In [178]:
# function to plot the generated images within a training epoch
def plot_gen_images(examples, epoch, report_fp_name, n_sample):

    # get important data for iterating
    example_size = examples.shape[0]
    og_shape = examples[0].shape
    rand_img = np.random.choice(example_size, size=n_sample*n_sample, replace=False) 
    # (0, example_size, size=n_sample*n_sample)

    # prep the figure
    fig, ax = plt.subplots(n_sample,n_sample, figsize=(20,20))
    fig.patch.set_facecolor("xkcd:white")

    # plot images
    for i in range(n_sample*n_sample):
        # define subplot
        plt.subplot(n_sample, n_sample, 1+i)

        # getting the images from sample
        rand_i = rand_img[i]
        gimg = examples[rand_i]
        gimg = inv_std_img(gimg, 0, 255, "ctr")

        # turn off axis
        plt.axis("off")
        plt.imshow(gimg) #, interpolation="nearest")

    # plot leyend
    fig.suptitle("GENERATED IMAGES", fontsize=50)
    fig.legend()

    # save plot to file
    plot_name = "GAN-Gen-img-epoch%03d" % int(epoch)
    plot_name = plot_name + ".png"
    fpn = os.path.join(report_fp_name, plot_name)
    plt.savefig(fpn)
    plt.close()

In [179]:
# create a line plot of loss for the gan and save to file
def plot_metrics(disr_hist, disf_hist, gan_hist, report_fp_name, epoch):

    # reporting results
    disr_hist = np.array(disr_hist)
    disf_hist = np.array(disf_hist)
    gan_hist = np.array(gan_hist)

    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,8))
    fig.patch.set_facecolor("xkcd:white")

    # loss
    ax1.plot(disr_hist[:,1], "royalblue", label="Loss: R-Dis")
    ax1.plot(disf_hist[:,1], "crimson", label="Loss: F-Dis")
    ax1.plot(gan_hist[:,1], "blueviolet", label="Loss: GAN/Gen")
    # ax1.plot(gan_hist[:], "blueviolet", label="Loss: GAN/Gen")

    # acc_
    ax2.plot(disr_hist[:,0], "royalblue", label="Acc: R-Dis")
    ax2.plot(disf_hist[:,0], "crimson", label="Acc: F-Dis")
    ax2.plot(gan_hist[:,0], "blueviolet", label="Acc: GAN/Gem")

    # plot leyend
    fig.suptitle("LEARNING BEHAVIOR", fontsize=20)
    ax1.grid(True)
    ax2.grid(True)
    ax1.set_title("Loss")
    ax2.set_title("Accuracy")
    ax1.set(xlabel = "Epoch [cycle]", ylabel = "Loss")
    ax2.set(xlabel = "Epoch [cycle]", ylabel = "Acc")
    fig.legend()

    # save plot to file
    plot_name = "GAN-learn-curve-epoch%03d" % int(epoch)
    plot_name = plot_name + ".png"
    fpn = os.path.join(report_fp_name, plot_name)
    plt.savefig(fpn)
    plt.close()

In [180]:
# function to calculate the loss and accuracy avg in multiple batchs of an epoch
def epoch_avg(log):
    loss, acc = None, None

    # if acc and loss are present to avg
    if type(log[0]) is list:
        if len(log) > 0:

            acc_list = list()
            loss_list = list()

            for l in log:
                ta = l[0]
                tl = l[1]

                acc_list.append(ta)
                loss_list.append(tl)

            loss, acc = mean(loss_list), mean(acc_list)
        return loss, acc
    
    else:
        # if only loss is present
        if len(log) > 0:

            loss_list = list()

            for l in log:
                loss_list.append(l)

            loss = mean(loss_list)
        return loss


In [181]:
# function to save model, needs the dirpath, the name and the datetime to save
def export_model(model, models_fp_name, filename, datetime):

    ss = True
    sln = True
    fext = "png"
    fpn = filename + "-" + datetime
    fpn = filename + "." + fext
    fpn = os.path.join(models_fp_name, fpn)
    plot_model(model, to_file=fpn, show_shapes=ss, show_layer_names=sln)

In [182]:
# function to format data to save in file
def format_metrics(disr_history, disf_history, gan_history):

    headers, data = None, None

    disr_hist = np.array(disr_history)
    disf_hist = np.array(disf_history)
    gan_hist = np.array(gan_history)

    # formating file headers
    headers = ["dis_loss_real", "dis_acc_real", "dis_loss_fake", "dis_acc_fake", "gen_gan_loss", "gen_gan_acc"]
    # headers = ["dis_loss_real", "dis_acc_real", "dis_loss_fake", "dis_acc_fake", "gen_gan_loss",] # "gen_gan_acc"]

    # formating fake discriminator train data
    drhl = disr_hist[:,1]
    drha = disr_hist[:,0]

    # formating real discrimintator train data
    dfhl = disf_hist[:,1]
    dfha = disf_hist[:,0]

    # formating gan/gen train data
    # gghl = gan_hist[:]# .flatten()
    gghl = gan_hist[:,1]
    ggha = gan_hist[:,0]

    # adding all formatted data into list
    data = np.column_stack((drhl, drha, dfhl, dfha, gghl, ggha))
    # data = np.column_stack((drhl, drha, dfhl, dfha, gghl)) #, ggha))

    return data, headers

In [183]:
# function to write data in csv file
def write_metrics(data, headers, report_fp_name, filename):

    # print(report_fp_name, filename)
    fpn = filename + "-train-history.csv"
    fpn = os.path.join(report_fp_name, fpn)

    history_df = pd.DataFrame(data, columns=headers)
    tdata = history_df.to_csv(
                            fpn,
                            sep=",",
                            index=False,
                            encoding="utf-8",
                            mode="w",
                            quoting=csv.QUOTE_ALL
                            )

In [184]:
# function to safe the loss/acc logs in training for the gan/gen/dis models
def save_metrics(disr_history, disf_history, gan_history, report_fp_name, filename):

    data, headers = format_metrics(disr_history, disf_history, gan_history)
    write_metrics(data, headers, report_fp_name, filename)

In [185]:
# function to know the time between epochs or batchs it return the new time for a new calculation
def lapse_time(last_time, epoch):

    now_time = datetime.datetime.now()
    deltatime = now_time - last_time
    deltatime = deltatime.total_seconds()
    deltatime = "%.2f" % deltatime
    msg = "Epoch:%3d " % int(epoch+1)
    msg = msg + "elapsed time: " + str(deltatime) + " [s]"
    print(msg)
    return now_time

In [186]:
# special function to train the GAN
# https://machinelearningmastery.com/how-to-develop-a-generative-adversarial-network-for-an-mnist-handwritten-digits-from-scratch-in-keras/
def train(gen_model, dis_model, gan_model, X_img, X_txt, y, labels, epochs, batch_size, save_intervas, fn_config):

    # sample shape
    txt_shape = X_txt[0].shape
    img_shape = X_img[0].shape
    cat_shape = y[0].shape
    label_shape = labels[0].shape

    # sample size
    txt_size = X_txt.shape[0]
    img_size = X_img.shape[0]
    cat_size = y.shape[0]
    label_size = labels.shape[0]

    # augmentation factor
    synth_batch = 1 # OJO!
    n = 3

    # model IO configuration
    model_fn_path = fn_config[0]
    report_fn_path = fn_config[1]
    dis_model_name = fn_config[2]
    gen_model_name = fn_config[3]
    gan_model_name = fn_config[4]

    # fake/real batch division
    half_batch = int(batch_size/2)
    batch_per_epoch = int(txt_size/batch_size)
    real_batch = int((batch_size*synth_batch)/2)
    # batch_per_epoch = int((txt_size*synth_batch)/batch_size)

	# prepare lists for storing stats each epoch
    # disf_hist, disr_hist, gen_hist, gan_hist = list(), list(), list(), list()
    disf_hist, disr_hist, gan_hist = list(), list(), list()
    train_time = None
    # iterating in training epochs:
    for ep in range(epochs+1):
        # epoch logs
        # ep_disf_hist, ep_disr_hist, ep_gen_hist, ep_gan_hist = list(), list(), list(), list()
        ep_disf_hist, ep_disr_hist, ep_gan_hist = list(), list(), list()
        train_time = datetime.datetime.now()

        # iterating over training batchs
        for batch in range(batch_per_epoch):

            # select real txt2img for discrimintator
            real_data = gen_real_samples(X_txt, X_img, y, labels, img_size, half_batch)
            # expand the training sample for the discriminator
            # real_data = expand_samples(real_data, synth_batch)
            # print(Xt_real.shape, Xi_real.shape, y_real.shape, yl_real.shape)

            # create false txt for txt2img for generator
            fake_data = gen_fake_samples(gen_model, txt_shape, cat_shape[0], label_shape[0], half_batch)
            # expand the training sample for the discriminator
            # fake_data = expand_samples(fake_data, synth_batch)

            # print(Xt_fake.shape, Xi_fake.shape, y_fake.shape, yl_fake.shape)
            # drift labels to confuse the model
            real_data, fake_data = drift_labels(real_data, fake_data, half_batch, 0.05)
    
            # real data asignation
            Xt_real = real_data[0]
            Xi_real = real_data[1]
            y_real = real_data[2]
            yl_real = real_data[3]

            # fake data asignation
            Xt_fake = fake_data[0]
            Xi_fake = fake_data[1]
            y_fake = fake_data[2]
            yl_fake = fake_data[3]

            # train for real samples batch
            dhr = dis_model.train_on_batch([Xi_real, yl_real], y_real)
            # train for fake samples batch
            dhf = dis_model.train_on_batch([Xi_fake, yl_fake], y_fake)

            # prepare noisy text of latent space as input for the generator
            Xt_gen = gen_latent_txt(txt_shape, batch_size)
            # create inverted category for the fake noisy text
            y_gen = get_fake_positive(cat_shape[0], batch_size)
            # create inverted labels for the fake noisy text
            yl_gen = get_fake_poslbls(label_shape[0], batch_size)
            # update the generator via the discriminator's error
            gh = gan_model.train_on_batch([Xt_gen, yl_gen], y_gen)
            # print("ojo GAN!", gh)

            ep_disr_hist.append(dhf)
            ep_disf_hist.append(dhr)
            # ep_gen_hist.append(gh)
            ep_gan_hist.append(gh)

			# print('>%d, %d/%d, dis_=%.3f, gen=%.3f' % (ep+1, batch+1, bat_per_epo, dis_history, gen_history))
            log_msg = ">>> Epoch: %d, B/Ep: %d/%d, Batch S: %d" % (ep+1, batch+1, batch_per_epoch, batch_size*synth_batch)
            log_msg = "%s -> [R-Dis loss: %.3f, acc: %.3f]" % (log_msg, dhr[0], dhr[1])
            log_msg = "%s || [F-Dis loss: %.3f, acc: %.3f]" % (log_msg, dhf[0], dhf[1])
            log_msg = "%s || [Gen loss: %.3f, acc: %.3f]" % (log_msg, gh[0], gh[1])
            # log_msg = "%s || [Gen loss: %.3f]" % (log_msg, gh) # [0], gh[1])
            print(log_msg)

        # record history for epoch
        disr_hist.append(epoch_avg(ep_disr_hist))
        disf_hist.append(epoch_avg(ep_disf_hist))
        # gen_hist.append(epoch_avg(ep_gen_hist))
        gan_hist.append(epoch_avg(ep_gan_hist))

		# evaluate the model performance sometimes
        if (ep) % save_intervas == 0:
            print("Epoch:", ep+1, "Saving the training progress...")
            test_model(ep, gen_model, dis_model, X_txt, X_img, y, labels, half_batch, report_fn_path, synth_batch)
            plot_metrics(disr_hist, disf_hist, gan_hist, report_fn_path, ep)
            save_metrics(disr_hist, disf_hist, gan_hist, report_fn_path, gan_model_name)

		# saving the model sometimes
        if (ep) % int(save_intervas*5) == 0:
            epoch_sufix = "-epoch%d" % int(ep)
            # epoch_sufix = "-last"
            epoch_sufix = str(epoch_sufix)
            dis_mn = dis_model_name + epoch_sufix
            gen_mn = gen_model_name + epoch_sufix
            gan_mn = gan_model_name + epoch_sufix

            dis_path = os.path.join(model_fn_path, "Dis")
            gen_path = os.path.join(model_fn_path, "Gen")
            gan_path = os.path.join(model_fn_path, "GAN")

            save_model(dis_model, dis_path, dis_mn)
            save_model(gen_model, gen_path, gen_mn)
            save_model(gan_model, gan_path, gan_mn)
        
        train_time = lapse_time(train_time, ep)

# EXEC SCRIPT

## Dataset prep

In [187]:
# variable definitions
# root folder
dataf = "Data"

# subfolder with predictions txt data
imagef = "Img"

# report subfolder
reportf = "Reports"

#  subfolder with the CSV files containing the ML pandas dataframe
trainf = "Train"
testf = "Test"

# subfolder for model IO
modelf = "Models"

# dataframe file extension
fext = "csv"

imgf = "jpg"

rgb_sufix = "rgb"
bw_sufix = "bw"

# standard sufix
stdprefix = "std-"

# ml model useful data
mltprefix = "ml-"

# report names
# timestamp = datetime.date.today().strftime("%d-%b-%Y")
timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

sample_sufix = "Small"
# sample_sufix = "Large"
# sample_sufix = "Paintings"
imgf_sufix = "Img-Data-"
text_sufix = "Text-Data-"

# std-VVG-Gallery-Text-Data-Paintings
gallery_prefix = "VVG-Gallery-"

# dataframe file name
text_fn = stdprefix + gallery_prefix + text_sufix + sample_sufix + "." + fext
imgf_fn = stdprefix + gallery_prefix + imgf_sufix + sample_sufix + "." + fext
valt_fn = "Validation-GAN-" + text_sufix + sample_sufix + "." + fext

# model names
dis_model_name = "VVG-Text2Img-CDiscriminator"
gen_model_name = "VVG-Text2Img-CGenerator"
gan_model_name = "VVG-Text2Img-CGAN"

# to continue training after stoping script
continue_training = True

# ramdom seed
randseed = 42

# sample distribution train vs test sample size
train_split = 0.80
test_split = 1.0 - train_split

# regex to know that column Im interested in
keeper_regex = r"(^ID$)|(^std_)"

imgt = rgb_sufix
# imgt = bw_sufix

# woring values for code
work_txtf, work_imgf, work_sufix, work_imgt = text_fn, imgf_fn, sample_sufix, imgt

print("=== working files ===")
print("\n", work_txtf, "\n", work_imgf, "\n", work_sufix, "\n", work_imgt, "\n", valt_fn)

=== working files ===

 std-VVG-Gallery-Text-Data-Small.csv 
 std-VVG-Gallery-Img-Data-Small.csv 
 Small 
 rgb 
 Validation-GAN-Text-Data-Small.csv


In [188]:
root_folder = os.getcwd()
root_folder = os.path.split(root_folder)[0]
root_folder = os.path.normpath(root_folder)
print(root_folder)

c:\Users\Felipe\Documents\GitHub\sa-artea\VVG-MLModel-Trainer


In [189]:
# variable reading
# dataframe filepath for texttual data
text_fn_path = os.path.join(root_folder, dataf, trainf, work_txtf)
print(text_fn_path, os.path.exists(text_fn_path))

# dataframe filepath for img data
img_fn_path = os.path.join(root_folder, dataf, trainf, work_imgf)
print(img_fn_path, os.path.exists(img_fn_path))

# dataframe filepath form GAN data
val_fn_path = os.path.join(root_folder, dataf, testf, valt_fn)
print(val_fn_path, os.path.exists(val_fn_path))

# filepath for the models
model_fn_path = os.path.join(root_folder, dataf, modelf)
print(model_fn_path, os.path.exists(model_fn_path))

# filepath for the reports
report_fn_path = os.path.join(root_folder, dataf, reportf)
print(report_fn_path, os.path.exists(report_fn_path))

c:\Users\Felipe\Documents\GitHub\sa-artea\VVG-MLModel-Trainer\Data\Train\std-VVG-Gallery-Text-Data-Small.csv True
c:\Users\Felipe\Documents\GitHub\sa-artea\VVG-MLModel-Trainer\Data\Train\std-VVG-Gallery-Img-Data-Small.csv True
c:\Users\Felipe\Documents\GitHub\sa-artea\VVG-MLModel-Trainer\Data\Test\Validation-GAN-Text-Data-Small.csv False
c:\Users\Felipe\Documents\GitHub\sa-artea\VVG-MLModel-Trainer\Data\Models True
c:\Users\Felipe\Documents\GitHub\sa-artea\VVG-MLModel-Trainer\Data\Reports True


In [190]:
# rading training data
# loading textual file
text_df = pd.read_csv(
                text_fn_path,
                sep=",",
                encoding="utf-8",
                engine="python",
            )
text_cols = text_df.columns.values

# loading image file
img_df = pd.read_csv(
                img_fn_path,
                sep=",",
                encoding="utf-8",
                engine="python",
            )
img_cols = img_df.columns.values

In [191]:
idx_cols = list()

for tcol in text_cols:
    if tcol in img_cols:
        idx_cols.append(tcol)
print(idx_cols)

source_df = pd.merge(text_df, img_df, how="inner", on=idx_cols)

['ID', 'F-number', 'JH-number', 'creator-date', 'creator-place', 'Dimensions', 'details', 'std_cat_creator-date', 'std_cat_creator-place', 'std_cat_Dimensions', 'std_cat_details']


In [192]:
# checking everything is allrigth
img_df = None
text_df = None
source_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 0 to 58
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID                     59 non-null     object
 1   F-number               59 non-null     object
 2   JH-number              59 non-null     object
 3   creator-date           59 non-null     object
 4   creator-place          59 non-null     object
 5   Dimensions             59 non-null     object
 6   details                59 non-null     object
 7   MUS_TEXT               59 non-null     object
 8   std_cat_creator-date   59 non-null     object
 9   std_cat_creator-place  59 non-null     object
 10  std_cat_Dimensions     59 non-null     object
 11  std_cat_details        59 non-null     object
 12  clr_tokens             59 non-null     object
 13  lemmas                 59 non-null     object
 14  bows_tokens            59 non-null     object
 15  idxs_tokens            59

In [193]:
source_df = source_df.set_index("ID")

In [194]:
# reading images from folder and loading images into df
# working variables
src_col = work_imgt + "_img"
tgt_col = work_imgt + "_img" + "_data"
work_shape = work_imgt + "_shape"
scale = 50
print(src_col, tgt_col)
source_df = get_images(root_folder, source_df, src_col, tgt_col, scale)

rgb_img rgb_img_data


In [195]:
# update image shape
source_df = update_shape(source_df, tgt_col, work_shape)

# data augmentation
# source_df = augment_images(source_df, src_col, tgt_col, 6)
# source_df.info()

In [196]:
# searching the biggest shape in the image files
print(work_shape)
shape_data = source_df[work_shape]
max_shape = get_mshape(shape_data, work_imgt)
print(max_shape)

rgb_shape
(400, 400, 3)


In [197]:
# padding training data according to max shape of the images in gallery
pad_prefix = "pad_"
conv_prefix = "cnn_"
src_col = work_imgt + "_img" + "_data"
tgt_col = pad_prefix + conv_prefix + src_col

print(src_col, tgt_col, work_imgt)
source_df = padding_images(source_df, src_col, tgt_col, max_shape, work_imgt)

rgb_img_data pad_cnn_rgb_img_data rgb


In [198]:
# reading images from folder and stadarizing images into df
# working variables
print("standarizing regular images...")
src_col = work_imgt + "_img" + "_data"
tgt_col = "std_" + src_col

# source_df = standarize_images(source_df, src_col, tgt_col)

standarizing regular images...


In [199]:
print("standarizing padded images...")
src_col = pad_prefix + conv_prefix + work_imgt + "_img" + "_data"
tgt_col = "std_" + src_col
print(src_col, tgt_col)

# std_opt = "std"
std_opt = "ctr"
source_df = standarize_images(source_df, src_col, tgt_col, work_imgt, std_opt)

standarizing padded images...
pad_cnn_rgb_img_data std_pad_cnn_rgb_img_data


In [200]:
# shuffle the DataFrame rows
source_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, s0004V1962r to d1125S2005
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   F-number                  59 non-null     object
 1   JH-number                 59 non-null     object
 2   creator-date              59 non-null     object
 3   creator-place             59 non-null     object
 4   Dimensions                59 non-null     object
 5   details                   59 non-null     object
 6   MUS_TEXT                  59 non-null     object
 7   std_cat_creator-date      59 non-null     object
 8   std_cat_creator-place     59 non-null     object
 9   std_cat_Dimensions        59 non-null     object
 10  std_cat_details           59 non-null     object
 11  clr_tokens                59 non-null     object
 12  lemmas                    59 non-null     object
 13  bows_tokens               59 non-null     object
 14  idxs_tokens    

In [201]:
# cleaning memory
gc.collect()

474113

In [202]:
# function to find a name of column names according to a regex
def get_keeper_cols(col_names, search_regex):
    ans = [i for i in col_names if re.search(search_regex, i)]
    return ans

In [203]:
# function to find the disperse columns in the df
def get_disperse_categories(src_df, keep_cols, max_dis, check_cols, ignore_col):

    ans = list()

    max_dis = 2
    tcount = 0

    while tcount < max_dis:
        for label_col in keep_columns:

            if label_col != ignore_col:

                label_count = src_df[label_col].value_counts(normalize=False)

                if tcount < label_count.shape[0] and (check_cols in label_col):
                    tcount = label_count.shape[0]
                    ans.append(label_col)
                # print("count values of", label_col, ":=", label_count.shape)#.__dict__)
        tcount = tcount + 1
    
    return ans

In [204]:
# function to remove the disperse columns from the interesting ones
def remove_disperse_categories(keep_columns, too_disperse):
    for too in too_disperse:
        keep_columns.remove(too)
    return keep_columns

In [205]:
def padding_corpus(train_df, dvector_col, pad_prefix):
    # getting the corpus dense vectors
    work_corpus = np.asarray(train_df[dvector_col], dtype="object")

    # converting list of list to array of array
    print("Original txt shape", work_corpus.shape)

    # padding the representation
    work_corpus = pad_sequences(work_corpus, dtype='object', padding="post")
    # print("Padded txt shape", work_corpus.shape)

    # creating the new column and saving padded data
    padded_col_dvector = pad_prefix + dvector_col

    # print(padded_col)
    train_df[padded_col_dvector] = list(work_corpus)
    print("Padded txt shape", work_corpus.shape)
    return train_df

In [206]:
def heat_categories(train_df, cat_cols, tgt_col):

    labels_data = train_df[cat_cols]
    labels_concat = list()

    # concatenating all category labels from dataframe
    for index, row in labels_data.iterrows():
        row = concat_labels(row, labels_cols)
        labels_concat.append(row)

    # print(len(labels_concat[0]), type(labels_concat[0]))
    # updating dataframe
    tcat_label_col = "std_cat_labels"
    train_df[tgt_col] = labels_concat

    return train_df

In [207]:
# function to adjust the textual data for the LSTM layers in the model
def format_corpus(corpus, timesteps, features):

    # preparation for reshape lstm model
    corpus = temporalize(corpus, timesteps)
    print(corpus.shape)

    corpus = corpus.reshape((corpus.shape[0], timesteps, features))
    print(corpus.shape)

    return corpus

In [208]:
# selecting data to train
# want to keep the columns starting with STD_
keep_columns = list(source_df.columns)
print("------ original input/interested columns ------")
print(keep_columns)

# create the columns Im interesting in
keep_columns = get_keeper_cols(keep_columns, keeper_regex)
# keep_columns = [i for i in df_columns if re.search(keeper_regex, i)]

print("\n\n------ Interesting columns ------")
print(keep_columns)

------ original input/interested columns ------
['F-number', 'JH-number', 'creator-date', 'creator-place', 'Dimensions', 'details', 'MUS_TEXT', 'std_cat_creator-date', 'std_cat_creator-place', 'std_cat_Dimensions', 'std_cat_details', 'clr_tokens', 'lemmas', 'bows_tokens', 'idxs_tokens', 'tfidf_tokens', 'std_dvec_tokens', 'rgb_img', 'bw_img', 'rgb_shape', 'bw_shape', 'rgb_img_data', 'pad_cnn_rgb_img_data', 'std_pad_cnn_rgb_img_data']


------ Interesting columns ------
['std_cat_creator-date', 'std_cat_creator-place', 'std_cat_Dimensions', 'std_cat_details', 'std_dvec_tokens', 'std_pad_cnn_rgb_img_data']


In [209]:
too_disperse = get_disperse_categories(source_df, keep_columns, 2, "std_cat_", "std_pad_cnn_rgb_img_data")
print(too_disperse)

['std_cat_creator-date', 'std_cat_Dimensions']


In [210]:
# creating the training dataframe
keep_columns = remove_disperse_categories(keep_columns, too_disperse)
# keep_columns.remove("ID")
print("------ Interesting columns ------")
print(keep_columns)

------ Interesting columns ------
['std_cat_creator-place', 'std_cat_details', 'std_dvec_tokens', 'std_pad_cnn_rgb_img_data']


In [211]:
# creating the training dataframe
train_df = pd.DataFrame(source_df, columns=keep_columns)

In [212]:
# shuffling the stuff
train_df = train_df.sample(frac = 1)
source_df = None
df_columns = list(train_df.columns)

In [213]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, d0158V1962 to d0031V1962v
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   std_cat_creator-place     59 non-null     object
 1   std_cat_details           59 non-null     object
 2   std_dvec_tokens           59 non-null     object
 3   std_pad_cnn_rgb_img_data  59 non-null     object
dtypes: object(4)
memory usage: 2.3+ KB


In [214]:
# getting the column with the relevant data to train
pad_regex = u"^std_pad_"
padimg_col = get_keeper_cols(df_columns, pad_regex)
padimg_col = padimg_col[0]
print("Padded image column in dataframe: ", str(padimg_col))

Padded image column in dataframe:  std_pad_cnn_rgb_img_data


In [215]:
# getting the column with the relevant data to train
dvec_regex = u"^std_dvec"
dvector_col = get_keeper_cols(df_columns, dvec_regex)
dvector_col = dvector_col[0]
print("Dense vector column in dataframe: ", str(dvector_col))

Dense vector column in dataframe:  std_dvec_tokens


In [216]:
# fix column data type
work_corpus = train_df[dvector_col]
work_corpus = format_dvector(work_corpus)

In [217]:
# changing type in dataframe
train_df[dvector_col] = work_corpus
work_corpus = None

In [218]:
# padding training data according to max length of text corpus
pad_prefix = "pad_"
recurrent_prefix = "lstm_"

train_df = padding_corpus(train_df, dvector_col, pad_prefix)

Original txt shape (59,)
Padded txt shape (59, 142)


In [219]:
regular_img_col = "std_" + work_imgt + "_img" + "_data"
padded_img_col = "std_" + pad_prefix + conv_prefix + work_imgt + "_img" + "_data"
padded_col_dvector = pad_prefix + dvector_col

In [220]:
# getting the columns with the relevant labels to predict
print(keep_columns)
cat_regex = u"^std_cat_"
labels_cols = get_keeper_cols(keep_columns, cat_regex)
print("Classifier trainable labels in dataframe: ", str(labels_cols))

# updating dataframe with hot/concatenated categories
tcat_label_col = "std_cat_labels"
print("categories heat column:", tcat_label_col)
train_df = heat_categories(train_df, labels_cols, tcat_label_col)

['std_cat_creator-place', 'std_cat_details', 'std_dvec_tokens', 'std_pad_cnn_rgb_img_data']
Classifier trainable labels in dataframe:  ['std_cat_creator-place', 'std_cat_details']
categories heat column: std_cat_labels


In [221]:
# getting the columns with the relevant labels to predict
print(keep_columns)
labels_cols = [i for i in keep_columns if re.search(u"^std_cat_", i)]
print("Trainable labels columns in dataframe: ", str(labels_cols))

labels_data = train_df[labels_cols]
labels_concat = list()

# concatenating all category labels from dataframe
for index, row in labels_data.iterrows():
    row = concat_labels(row, labels_cols)
    labels_concat.append(row)

['std_cat_creator-place', 'std_cat_details', 'std_dvec_tokens', 'std_pad_cnn_rgb_img_data']
Trainable labels columns in dataframe:  ['std_cat_creator-place', 'std_cat_details']


In [222]:
text_lstm_col = padded_col_dvector
print(text_lstm_col)

pad_std_dvec_tokens


In [223]:
working_img_col = padded_img_col
# working_img_col = regular_img_col
print(working_img_col)

std_pad_cnn_rgb_img_data


In [224]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, d0158V1962 to d0031V1962v
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   std_cat_creator-place     59 non-null     object
 1   std_cat_details           59 non-null     object
 2   std_dvec_tokens           59 non-null     object
 3   std_pad_cnn_rgb_img_data  59 non-null     object
 4   pad_std_dvec_tokens       59 non-null     object
 5   std_cat_labels            59 non-null     object
dtypes: object(6)
memory usage: 3.2+ KB


In [225]:
gc.collect()

15

In [226]:
# creating Train/Test sample
# getting the X, y to train, as is autoencoder both are the same
og_shape = train_df[working_img_col][0].shape# y[0].shape
X_img_len = train_df[working_img_col].shape[0] #y.shape[0]
print(X_img_len, og_shape)

X_img = None

for img in train_df[working_img_col]:

    if X_img is None:
        img = np.expand_dims(img, axis=0)
        X_img = img
    else:
        img = np.expand_dims(img, axis=0)
        X_img = np.concatenate((X_img, img), axis=0)

print("final X_img shape", X_img.shape)
# y.shape = (1899, 800, 800, 3)

59 (400, 400, 3)
final X_img shape (59, 400, 400, 3)


In [227]:
print(type(X_img[0]))
print(type(X_img[0][0]))
print(X_img[1].shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(400, 400, 3)


In [228]:
if len(X_img.shape) == 3:
    X_img = X_img[:,:,:,np.newaxis]

In [229]:
# y = train_df[working_img_col]
# y = np.expand_dims(y, axis=0)
y_labels = np.asarray([np.asarray(j, dtype="object") for j in train_df[tcat_label_col]], dtype="object")
print("y shape", y_labels.shape)

y shape (59, 16)


In [230]:
y = np.ones((y_labels.shape[0],1)).astype("float32")
print("y shape", y.shape)

y shape (59, 1)


In [231]:
print("y classification category")
print(type(y[0]))
print(type(y[0][0]))
print(y[1].shape)

print("y labels category")
print(type(y_labels[0]))
print(type(y_labels[0][0]))
print(y_labels[1].shape)

y classification category
<class 'numpy.ndarray'>
<class 'numpy.float32'>
(1,)
y labels category
<class 'numpy.ndarray'>
<class 'float'>
(16,)


In [232]:
# creating Train/Test sample
# getting the X, y to train, as is autoencoder both are the same
X_txt = np.asarray([np.asarray(i, dtype="object") for i in train_df[text_lstm_col]], dtype="object")
# X = np.array(train_df[text_lstm_col]).astype("object")
# X = train_df[text_lstm_col]
print("final X_lstm shape", X_txt.shape)

final X_lstm shape (59, 142)


In [233]:
print(type(X_txt[0]))
print(type(X_txt[0][0]))
print(X_txt[1].shape)

<class 'numpy.ndarray'>
<class 'float'>
(142,)


In [234]:
# timestep is the memory of what i read, this is the longest sentence I can remember in the short term
# neet to look for the best option, in small the max is 15
timesteps = 15

# features is the max length in the corpus, after padding!!!!
features = X_txt[0].shape[0]
print(timesteps, features)

15 142


In [235]:
X_txt = format_corpus(X_txt, timesteps, features)

(43, 15, 1, 142)
(43, 15, 142)


In [236]:
print(X_txt.shape)

(43, 15, 142)


In [237]:
diff_txt = y.shape[0] - X_txt.shape[0]
print(diff_txt)

16


In [238]:
Xa = X_txt[-diff_txt:]
X_txt = np.append(X_txt, Xa, axis=0)
print(X_txt.shape)
Xa = None

(59, 15, 142)


In [239]:
print(X_txt.shape)
print(X_img.shape)
print(y.shape)
print(y_labels.shape)

(59, 15, 142)
(59, 400, 400, 3)
(59, 1)
(59, 16)


In [240]:
print(X_txt[0].shape)
print(X_img[0].shape)
print(y[0].shape)
print(y_labels[0].shape)
txt_og_shape = X_txt[0].shape
img_og_shape = X_img[0].shape
cat_og_shape = y[0].shape
lab_og_shape = y_labels[0].shape

(15, 142)
(400, 400, 3)
(1,)
(16,)


In [241]:
# Xt = X_txt # np.array(X).astype("object")
# Xi = X_img
# yt = y # np.array(y).astype("object")
# # ya = y[0:timesteps]
# train_df = None

In [242]:
gc.collect()

75

## ML Model Definition

In [243]:
# keras model definition for the GAN conditional discriminator
def create_cdiscriminator(img_shape, txt_shape, n_labels, layer_cfg):

    # model layer configuration!!!! get from layer_cfg!!!
    dis_ldrop = layer_cfg[0]
    dis_filters = layer_cfg[1]
    ksize = layer_cfg[2]
    stsize = layer_cfg[3]
    in_dis_actf = layer_cfg[4]
    hid_ly_actf = layer_cfg[5]
    psize = layer_cfg[6]
    pad = layer_cfg[7]
    dis_midn = layer_cfg[8]
    out_dis = layer_cfg[9]
    out_dis_act = layer_cfg[10]
    gen_reshape = layer_cfg[11]

    # scalar to fit the image dimensions in a dense layer
    img_label_neurons = img_shape[0]*img_shape[1]*img_shape[2]

    # LABEL IMG LAYERS
    # label inpuy
    in_labels = Input(shape=(n_labels,), 
                        dtype="float32", 
                        name="ImgLabelsIn")

    # embedding categorical textual input
    embed1 = Embedding(txt_shape[0], txt_shape[1], 
                        input_length=n_labels, 
                        name="ImgLblEmb_1")(in_labels)
    # flat layer
    label1 = Flatten(name="ImgLblFlat")(embed1)
    label2 = Dense(dis_midn, activation=hid_ly_actf, 
                    name="ImgLblDense_1")(label1)
    # reshape layer
    label3 = Reshape(gen_reshape, name="ImgLblReshape")(label2)

    # upsampling conv2D layers
    label4 = Conv2D(int(dis_filters/8), kernel_size=ksize, 
                        strides=stsize, activation=hid_ly_actf, 
                        padding=pad, name="ImglblConv2D_1")(label3)
    label5 = UpSampling2D(psize, name="ImgLblUpsam_1")(label4)

    # std and drop layer
    batch5 = BatchNormalization(name="ImgBaNorm_1")(label5)
    drop5 = Dropout(dis_ldrop, name="ImgDrop_1")(batch5)

    # upsampling conv2D layers
    label6 = Conv2D(int(dis_filters/4), kernel_size=ksize, 
                        strides=stsize, activation=hid_ly_actf, 
                        padding=pad, name="ImglblConv2D_2")(drop5)
    label7 = UpSampling2D(psize, name="ImgLblUpsam_2")(label6)

    # upsampling conv2D layers
    label8 = Conv2D(int(dis_filters/2), kernel_size=ksize, 
                        strides=stsize, activation=hid_ly_actf, 
                        padding=pad, name="ImglblConv2D_3")(label7)
    label9 = UpSampling2D(psize, name="ImgLblUpsam_3")(label8)

    # std and drop layer
    batch6 = BatchNormalization(name="ImgBaNorm_2")(label9)
    drop6 = Dropout(dis_ldrop, name="ImgDrop_2")(batch6)

    label10 = Conv2D(img_shape[2], kernel_size=ksize, 
                        strides=stsize, activation=hid_ly_actf, 
                        padding=pad, name="ImglblOut")(drop6)

    # MERGING DISCRIMINATOR + LABEL MODEL
    # input layer
    in_layer = Input(shape=(img_shape[0], img_shape[1], img_shape[2]), name="DisIn")

    concat1 = Concatenate(axis=-1, name="DisConcat_1")([in_layer, label10])

    # DISCRIMINATOR LAYERS
    # intermediate conv layer
    conv1 = Conv2D(int(dis_filters), kernel_size=ksize, 
                    padding=pad, activation=in_dis_actf, 
                    name="DisConv2D_1")(concat1)
    pool1 = MaxPooling2D(pool_size=psize, name="DisMax2D_1")(conv1)

    # std and drop layer
    batch1 = BatchNormalization(name="DisBaNorm_1")(pool1)
    drop1 = Dropout(dis_ldrop, name="DisDrop_1")(batch1)

    # intermediate conv layer
    conv2 = Conv2D(int(dis_filters/2), kernel_size=ksize, 
                    padding=pad, activation=hid_ly_actf, 
                    name="DisConv2D_2")(drop1)
    pool2 = MaxPooling2D(pool_size=psize, name="DisMax2D_2")(conv2)

    # intermediate conv layer
    conv3 = Conv2D(int(dis_filters/4), kernel_size=ksize, 
                    padding=pad, activation=hid_ly_actf, 
                    name="DisConv2D_3")(pool2)
    pool3 = MaxPooling2D(pool_size=psize, name="DisMax2D_3")(conv3)

    # intermediate conv layer
    conv4 = Conv2D(int(dis_filters/8), kernel_size=ksize, 
                    padding=pad, activation=hid_ly_actf, 
                    name="DisConv2D_4")(pool3)
    pool4 = MaxPooling2D(pool_size=psize, name="DisMax2D_4")(conv4)

    # std and drop layer
    batch2 = BatchNormalization(name="DisBaNorm_2")(pool4)
    drop2 = Dropout(dis_ldrop, name="DisDrop_2")(batch2)

    # flatten from 2D to 1D
    flat = Flatten(name="DisMidFlat")(pool4)

    # dense classifier layers
    hidden1 = Dense(int(dis_midn), activation=hid_ly_actf, name="DisDense_1")(flat)
    hidden2 = Dense(int(dis_midn/2), activation=hid_ly_actf, name="DisDense_2")(hidden1)
    # drop layer
    drop3 = Dropout(dis_ldrop, name="DisDrop_3")(hidden2)

    # dense classifier layers
    hidden3 = Dense(int(dis_midn/4), activation=hid_ly_actf, name="DisDense_3")(drop3)
    hidden4 = Dense(int(dis_midn/8), activation=hid_ly_actf, name="DisDense_4")(hidden3)
    # drop layer
    drop4 = Dropout(dis_ldrop, name="DisDrop_4")(hidden4)

    # output layer
    out_layer = Dense(out_dis, activation=out_dis_act, name="DisOut")(drop4)

    # model definition
    dis_model = Model(inputs=[in_layer, in_labels], outputs=out_layer)

    return dis_model

In [244]:
def create_cdiscriminator_v2(img_shape, txt_shape, n_labels, layer_cfg):

    # model layer configuration!!!! get from layer_cfg!!!
    dis_ldrop = layer_cfg[0]
    dis_filters = layer_cfg[1]
    ksize = layer_cfg[2]
    stsize = layer_cfg[3]
    in_dis_actf = layer_cfg[4]
    hid_ly_actf = layer_cfg[5]
    psize = layer_cfg[6]
    pad = layer_cfg[7]
    dis_midn = layer_cfg[8]
    out_dis = layer_cfg[9]
    out_dis_act = layer_cfg[10]
    gen_reshape = layer_cfg[11]

    # scalar to fit the image dimensions in a dense layer
    img_label_neurons = img_shape[0]*img_shape[1]*img_shape[2]

    # LABEL IMG LAYERS
    # label inpuy
    in_labels = Input(shape=(n_labels,), 
                        dtype="float32", 
                        name="ImgLabelsIn")

    # embedding categorical textual input
    embed1 = Embedding(txt_shape[0], txt_shape[1], 
                        input_length=n_labels, 
                        name="ImgLblEmb_1")(in_labels)
    # flat layer
    label1 = Flatten(name="ImgLblFlat")(embed1)
    label2 = Dense(dis_midn, activation=hid_ly_actf, 
                    name="ImgLblDense_1")(label1)
    # reshape layer
    label3 = Reshape(gen_reshape, name="ImgLblReshape")(label2)

    # transpose conv2D layers
    label4 = Conv2DTranspose(int(dis_filters/8), kernel_size=ksize, 
                                strides=stsize, activation=hid_ly_actf, 
                                padding=pad, name="ImglblConv2D_1")(label3)
    # label5 = UpSampling2D(psize, name="ImgLblUpsam_1")(label4)

    # std and drop layer
    batch5 = BatchNormalization(name="ImgBaNorm_1")(label4)
    drop5 = Dropout(dis_ldrop, name="ImgDrop_1")(batch5)

    # trnaspose conv2D layers
    label6 = Conv2DTranspose(int(dis_filters/4), kernel_size=ksize, 
                                strides=stsize, activation="tanh", 
                                padding=pad, name="ImglblConv2D_2")(drop5)
    # label7 = UpSampling2D(psize, name="ImgLblUpsam_2")(label6)

    # std and drop layer
    batch6 = BatchNormalization(name="ImgBaNorm_2")(label6)
    drop6 = Dropout(dis_ldrop, name="ImgDrop_2")(batch6)

    # conditional layer output
    label10 = Conv2DTranspose(img_shape[2], kernel_size=ksize, 
                                strides=stsize, activation=hid_ly_actf, 
                                padding=pad, name="ImglblOut")(drop6)

    # MERGING DISCRIMINATOR + LABEL MODEL
    # input layer
    in_layer = Input(shape=(img_shape[0], img_shape[1], img_shape[2]), name="DisIn")

    concat1 = Concatenate(axis=-1, name="DisConcat_1")([in_layer, label10])

    # DISCRIMINATOR LAYERS
    # intermediate conv layer
    conv1 = Conv2D(int(dis_filters), kernel_size=ksize, 
                    padding=pad, activation=in_dis_actf, 
                    strides=stsize, name="DisConv2D_1")(concat1)
    # pool1 = MaxPooling2D(pool_size=psize, name="DisMax2D_1")(conv1)

    # std and drop layer
    batch1 = BatchNormalization(name="DisBaNorm_1")(conv1)
    drop1 = Dropout(dis_ldrop, name="DisDrop_1")(batch1)

    # intermediate conv layer
    conv2 = Conv2D(int(dis_filters/2), kernel_size=ksize, 
                    padding=pad, activation=hid_ly_actf, 
                    strides=stsize, name="DisConv2D_2")(drop1)
    # pool2 = MaxPooling2D(pool_size=psize, name="DisMax2D_2")(conv2)

    # intermediate conv layer
    conv3 = Conv2D(int(dis_filters/4), kernel_size=ksize, 
                    padding=pad, activation=hid_ly_actf, 
                    strides=stsize, name="DisConv2D_3")(conv2)
    # pool3 = MaxPooling2D(pool_size=psize, name="DisMax2D_3")(conv3)

    # intermediate conv layer
    conv4 = Conv2D(int(dis_filters/8), kernel_size=ksize, 
                    padding=pad, activation=hid_ly_actf, 
                    strides=stsize, name="DisConv2D_4")(conv3)
    # pool4 = MaxPooling2D(pool_size=psize, name="DisMax2D_4")(conv4)

    # std and drop layer
    batch2 = BatchNormalization(name="DisBaNorm_2")(conv4)
    drop2 = Dropout(dis_ldrop, name="DisDrop_2")(batch2)

    # flatten from 2D to 1D
    flat = Flatten(name="DisMidFlat")(drop2)

    # dense classifier layers
    hidden1 = Dense(int(dis_midn), activation=hid_ly_actf, name="DisDense_1")(flat)
    hidden2 = Dense(int(dis_midn/2), activation=hid_ly_actf, name="DisDense_2")(hidden1)
    # drop layer
    drop3 = Dropout(dis_ldrop, name="DisDrop_3")(hidden2)

    # dense classifier layers
    hidden3 = Dense(int(dis_midn/4), activation=hid_ly_actf, name="DisDense_3")(drop3)
    hidden4 = Dense(int(dis_midn/8), activation=hid_ly_actf, name="DisDense_4")(hidden3)
    # drop layer
    drop4 = Dropout(dis_ldrop, name="DisDrop_4")(hidden4)

    # dense classifier layers
    hidden5 = Dense(int(dis_midn/16), activation=hid_ly_actf, name="DisDense_5")(drop4)
    hidden6 = Dense(int(dis_midn/32), activation=hid_ly_actf, name="DisDense_6")(hidden5)

    # output layer
    out_layer = Dense(out_dis, activation=out_dis_act, name="DisOut")(hidden6)

    # model definition
    dis_model = Model(inputs=[in_layer, in_labels], outputs=out_layer)

    return dis_model

In [245]:
def create_cgenerator(img_shape, txt_shape, n_labels, layer_cfg):

    # features for text
    memory = txt_shape[0]
    features = txt_shape[1]

    # features for layers
    lstm_units = layer_cfg[0]
    gen_ldrop = layer_cfg[1]
    gen_filters = layer_cfg[2]
    ksize = layer_cfg[3]
    stsize = layer_cfg[4]
    in_gen_actf = layer_cfg[5]
    hid_ly_actf = layer_cfg[6]
    psize = layer_cfg[7]
    pad = layer_cfg[8]
    gen_midn = layer_cfg[9]
    out_gen_act = layer_cfg[10]
    rs = layer_cfg[11]
    gen_reshape = layer_cfg[12]

    # TXT LABEL LAYERS
    # label inpuy
    in_labels = Input(shape=(n_labels,), dtype="float32", name="TxtLabelsIn")

    # embedding categorical textual input
    embed1 = Embedding(txt_shape[0], txt_shape[1], input_length=n_labels, name="TxtLblEmb_1")(in_labels)
    # flat layer
    label1 = Flatten(name="TxtLblFlat")(embed1)
    # dense layers
    label2 = Dense(dis_midn, activation=hid_ly_actf, name="LblDense_1")(label1)
    # std and drop layer
    batch5 = BatchNormalization(name="TxtBaNorm_1")(label2)
    drop5 = Dropout(dis_ldrop, name="TxtDrop_1")(batch5)

    label3 = Dense(int(txt_shape[0]*txt_shape[1]), activation="sigmoid", name="TxtLblDense_2")(drop5)
    # reshape layer
    label4 = Reshape(txt_shape, name="TxtLblOut")(label3)


    # GENERATOR LAYERS
    # input layer
    in_layer = Input(shape=(txt_shape[0], txt_shape[1]), name="GenIn")

    # concat layer
    concat1 = Concatenate(axis=-1, name="GenConcat_1")([in_layer, label4])

    # masking input text
    mask = Masking(mask_value=0.0, input_shape=(memory, features), 
                    name = "GenMaskIn")(concat1) # concat1

    # intermediate recurrent layer
    lstm1 = LSTM(lstm_units, activation=in_gen_actf, 
                    input_shape=(memory, features), 
                    return_sequences=rs, name="GenLSTM_1")(mask)
    for img
    # std and drop layer
    batch1 = BatchNormalization(name="GenBaNorm_1")(lstm1)
    drop1 = Dropout(dis_ldrop, name="GenDrop_1")(batch1)

    # intermediate recurrent layer
    lstm2 = LSTM(int(lstm_units/2), activation=in_gen_actf, 
                    input_shape=(memory, features), 
                    return_sequences=rs, name="GenLSTM_2")(drop1)
    # intermediate recurrent layer
    lstm3 = LSTM(int(lstm_units/4), activation=in_gen_actf, 
                    input_shape=(memory, features), 
                    return_sequences=rs, name="GenLSTM_3")(lstm2)

    # std and drop layer
    batch2 = BatchNormalization(name="GenBaNorm_2")(lstm3)
    drop2 = Dropout(dis_ldrop, name="GenDrop_2")(batch2)

    # flatten from 2D to 1D
    flat = Flatten(name="GenMidFlat")(drop2)
    # dense classifier layers
    mid = Dense(int(dis_midn), activation=hid_ly_actf, 
                name="GenGenDense_1")(flat)
    # reshape layer
    gen1 = Reshape(gen_reshape, name="GenReshape")(mid)

    # upsampling conv2D layers
    gen2 = Conv2D(int(gen_filters/8), kernel_size=ksize, 
                    strides=stsize, activation=hid_ly_actf, 
                    padding=pad, name="GenConv2D_1")(gen1)
    gen3 = UpSampling2D(psize, name="GenUpsam_1")(gen2)

    # std and drop layer
    batch3 = BatchNormalization(name="GenBaNorm_3")(gen3)
    drop3 = Dropout(gen_ldrop, name="GenDrop_3")(batch3)

    # upsampling conv2D layers
    gen4 = Conv2D(int(gen_filters/4), kernel_size=ksize, 
                    strides=stsize, activation=hid_ly_actf, 
                    padding=pad, name="GenConv2D_2")(drop3)
    gen5 = UpSampling2D(psize, name="GenUpsam_2")(gen4)

    # upsampling conv2D layers
    gen6 = Conv2D(int(gen_filters/2), kernel_size=ksize, 
                    strides=stsize, activation=hid_ly_actf, 
                    padding=pad, name="GenConv2D_3")(gen5)
    gen7 = UpSampling2D(psize, name="GenUpsam_3")(gen6)

    # std and drop layer
    batch4 = BatchNormalization(name="GenBaNorm_4")(gen7)
    drop4 = Dropout(gen_ldrop, name="GenDrop_4")(batch4)

    # output layer
    out_layer = Conv2D(img_shape[2], kernel_size=ksize, 
                        strides=stsize, activation=hid_ly_actf, 
                        padding=pad, input_shape=img_shape, 
                        name="GenOut")(drop4)

    # model definition
    gen_model = Model(inputs=[in_layer, in_labels], outputs=out_layer)
    # gen_model = Model(inputs=in_layer, outputs=out_layer)

    return gen_model

In [246]:
def create_cgenerator_v2(img_shape, txt_shape, n_labels, layer_cfg):

    # features for text
    memory = txt_shape[0]
    features = txt_shape[1]

    # features for layers
    lstm_units = layer_cfg[0]
    gen_ldrop = layer_cfg[1]
    gen_filters = layer_cfg[2]
    ksize = layer_cfg[3]
    stsize = layer_cfg[4]
    in_gen_actf = layer_cfg[5]
    hid_ly_actf = layer_cfg[6]
    psize = layer_cfg[7]
    pad = layer_cfg[8]
    gen_midn = layer_cfg[9]
    out_gen_act = layer_cfg[10]
    rs = layer_cfg[11]
    gen_reshape = layer_cfg[12]

    # TXT LABEL LAYERS
    # label inpuy
    in_labels = Input(shape=(n_labels,), dtype="float32", name="TxtLabelsIn")

    # embedding categorical textual input
    embed1 = Embedding(txt_shape[0], txt_shape[1], input_length=n_labels, name="TxtLblEmb_1")(in_labels)
    # flat layer
    label1 = Flatten(name="TxtLblFlat")(embed1)
    # dense layers
    label2 = Dense(dis_midn, activation=hid_ly_actf, name="LblDense_1")(label1)
    # std and drop layer
    batch5 = BatchNormalization(name="TxtBaNorm_1")(label2)
    drop5 = Dropout(dis_ldrop, name="TxtDrop_1")(batch5)

    label3 = Dense(int(txt_shape[0]*txt_shape[1]), activation=hid_ly_actf, name="TxtLblDense_2")(drop5)
    # reshape layer
    label4 = Reshape(txt_shape, name="TxtLblOut")(label3)

    # GENERATOR LAYERS
    # input layer
    in_layer = Input(shape=(txt_shape[0], txt_shape[1]), name="GenIn")

    # concat layer
    concat1 = Concatenate(axis=-1, name="GenConcat_1")([in_layer, label4])

    # masking input text
    mask = Masking(mask_value=0.0, input_shape=(memory, features), 
                    name = "GenMaskIn")(concat1) # concat1

    # intermediate recurrent layer
    lstm1 = LSTM(lstm_units, activation=in_gen_actf, 
                    input_shape=(memory, features), 
                    return_sequences=rs, name="GenLSTM_1")(mask)

    # std and drop layer
    batch1 = BatchNormalization(name="GenBaNorm_1")(lstm1)
    drop1 = Dropout(dis_ldrop, name="GenDrop_1")(batch1)

    # intermediate recurrent layer
    lstm2 = LSTM(int(lstm_units/2), activation=in_gen_actf, 
                    input_shape=(memory, features), 
                    return_sequences=rs, name="GenLSTM_2")(drop1)
    # intermediate recurrent layer
    # lstm3 = LSTM(int(lstm_units/4), activation=in_gen_actf, 
    #                 input_shape=(memory, features), 
    #                 return_sequences=rs, name="GenLSTM_3")(lstm2)

    # std and drop layer
    batch2 = BatchNormalization(name="GenBaNorm_2")(lstm2)
    drop2 = Dropout(dis_ldrop, name="GenDrop_2")(batch2)

    # flatten from 2D to 1D
    flat = Flatten(name="GenMidFlat")(drop2)
    # dense classifier layers
    mid = Dense(int(dis_midn), activation=hid_ly_actf, 
                name="GenGenDense_1")(flat)
    # reshape layer
    gen1 = Reshape(gen_reshape, name="GenReshape")(mid)

    # transpose conv2D layers
    gen2 = Conv2DTranspose(int(gen_filters/8), kernel_size=ksize, 
                            strides=stsize, activation=hid_ly_actf, 
                            padding=pad, name="GenConv2D_1")(gen1)
    # gen3 = UpSampling2D(psize, name="GenUpsam_1")(gen2)

    # std and drop layer
    batch3 = BatchNormalization(name="GenBaNorm_3")(gen2)
    drop3 = Dropout(gen_ldrop, name="GenDrop_3")(batch3)

    # upsampling conv2D layers
    gen4 = Conv2DTranspose(int(gen_filters/4), kernel_size=ksize, 
                            strides=stsize, activation=hid_ly_actf, 
                            padding=pad, name="GenConv2D_2")(drop3)
    # gen5 = UpSampling2D(psize, name="GenUpsam_2")(gen4)

    # upsampling conv2D layers
    gen6 = Conv2DTranspose(int(gen_filters/2), kernel_size=ksize, 
                            strides=stsize, activation=hid_ly_actf, 
                            padding=pad, name="GenConv2D_3")(gen4)
    # gen7 = UpSampling2D(psize, name="GenUpsam_3")(gen6)

    # std and drop layer
    batch4 = BatchNormalization(name="GenBaNorm_4")(gen6)
    drop4 = Dropout(gen_ldrop, name="GenDrop_4")(batch4)

    # output layer
    out_layer = Conv2D(img_shape[2], kernel_size=(3,3), 
                        strides=(1,1), activation=hid_ly_actf, 
                        padding=pad, input_shape=img_shape, 
                        name="GenOut")(drop4)

    # model definition
    gen_model = Model(inputs=[in_layer, in_labels], outputs=out_layer)
    # gen_model = Model(inputs=in_layer, outputs=out_layer)

    return gen_model

In [247]:
# number of neurons or processing units in LSTM(())
# the number is because of good practices for NLP
# min 200 max 500, normaly 300 (related to the semantic number of themes)
# 120 for now in this test
lstm_units = 500
print("Generator LSMT neurons:", lstm_units)

# timestep is 1 because you read a word at a time
memory = timesteps
print("Generator LSTM memory span:", memory)
# configuration to remember previous recurrent layer
rs = True

# features is the max length in the corpus, after padding!!!!
# print(X_train.shape)
features = X_txt.shape[2]
print("Generator LSTM learning features:", features)

# batch size
bs = 32
print("Discriminator & Generator learning batch size:", bs)

# number of filters or processing units in CNN
# the number is because of good practices from computer vision
# min 8 max 64, normaly 32 (related to the size of the images)
# 16 for now in this test

# imgage filters
# filters = 16
# filters = 32
# filters = 64
# filters = 128
gen_filters = 128
dis_filters = 64

print("Generator CNN filter number:", gen_filters)
print("Discriminator CNN filter number:", dis_filters)

disin_shape = X_img[0].shape
genout_shape = X_img[0].shape
genin_shape = X_txt[0].shape
# in_shape = (None, None, 1)
# in_shape = (794, 794, 3)
print("Discriminator Input Img shape:", disin_shape)
print("Generator Output Img shape:", genout_shape)
print("Genewrator Input Text shape:", X_txt[0].shape)

ksize = (3,3) 
ksize_s = (4,4)
stsize = (1,1)
stsize_s = (2,2)
# psize = (5,5)
psize = (2,2)

print("Discriminator & Generator CNN kernel size:", ksize)
print("Discriminator & Generator CNN pad size:", psize)

# neurons/processing units size in the dense layer (THIS SHOULD BE SOM!!!!)
gen_midn =  50*50*3 # 100*100*3 # 
gen_reshape = (50,50,3) # (100,100,3) # 
print("Generator Dense middle neurons:", gen_midn)
# dn2 = len(XB_set[0])*SECURITY_FACTOR

# numero de neuronas de salida
# out_shape = X_train[0].shape
# out_shape = (None, None, 3)
# out_shape = in_shape
# out_dis = y[0].shape[0]
n_labels = y_labels[0].shape[0]
out_dis = y[0].shape[0]
print("Discriminator Output prediction labels:", n_labels)
print("Discriminator Output prediction Classification:", out_dis)

channels = img_og_shape[2]
# channels = 8
# dis_midn = filters*out_dis*channels*15*5
dis_midn = 50*50*3
print("Discriminator Dense middle neurons:", dis_midn)

# axtivation functions
in_dis_actf = LeakyReLU(alpha=0.2) # "relu"
in_gen_actf = LeakyReLU(alpha=0.2) # "relu"
hid_ly_actf = LeakyReLU(alpha=0.2) # "relu",
# in_dis_actf = 0.2 # "relu"
# in_gen_actf = 0.2 # "relu"
# hid_ly_actf = 0.2 # "relu",

out_dis_act = "sigmoid" # "softmax" #
out_gen_act = "tanh" # "softmax" #

# loss percentage
dis_ldrop = 0.4
gen_ldrop = 0.2

# padding policy
pad = "same"

# random seed
randseed = 42

# parameters to compile model
# loss function
# ls = "mean_squared_error"
# ls = "categorical_crossentropy"
ls = "binary_crossentropy"

##########################################
# discriminator optimization function
# Adam option 0.0002, 0.0003, 0.0004
dis_opti = Adam(learning_rate=0.0003, beta_1=0.5) #, amsgrad=True)

# Adadelta option
# dis_opti = Adadelta(learning_rate=0.00020)

# Adagrad option
# dis_opti = Adagrad(learning_rate=0.00020, momentum=0.5)

##########################################
# gan/genenerator optimization function
# Adam option
gan_opti = Adam(learning_rate=0.0002, beta_1=0.5) #, amsgrad=True)

# Adadelta option
# gan_opti = Adadelta(learning_rate=0.00030)

# Adagrad option
# gan_opti = Adagrad(learning_rate=0.00030, momentum=0.5)

# SGD option
# gan_opti = SGD(learning_rate=0.00020, momentum=0.5)

# evaluation score
# met = ["categorical_accuracy"]
met = ["accuracy"]

# parameters to exeute training
# verbose mode
ver = 0
# training epocha
epo = 500
print("training epochs:", epo)

Generator LSMT neurons: 500
Generator LSTM memory span: 15
Generator LSTM learning features: 142
Discriminator & Generator learning batch size: 32
Generator CNN filter number: 128
Discriminator CNN filter number: 64
Discriminator Input Img shape: (400, 400, 3)
Generator Output Img shape: (400, 400, 3)
Genewrator Input Text shape: (15, 142)
Discriminator & Generator CNN kernel size: (3, 3)
Discriminator & Generator CNN pad size: (2, 2)
Generator Dense middle neurons: 7500
Discriminator Output prediction labels: 16
Discriminator Output prediction Classification: 1
Discriminator Dense middle neurons: 7500
training epochs: 500


In [248]:
dis_layer_cfg = (
    dis_ldrop,
    dis_filters,
    ksize_s,
    stsize_s,
    # ksize,
    # stsize,
    in_dis_actf,
    hid_ly_actf,
    psize,
    pad,
    dis_midn,
    out_dis,
    out_dis_act,
    gen_reshape,
)

print(dis_layer_cfg)

(0.4, 64, (4, 4), (2, 2), <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x000001E46C77F910>, <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x000001E46C77F070>, (2, 2), 'same', 7500, 1, 'sigmoid', (50, 50, 3))


In [249]:
cdis_model = create_cdiscriminator_v2(disin_shape, genin_shape, n_labels, dis_layer_cfg)

In [250]:
print("CGAN Discriminator Definition")
# dis_model = Sequential(slim_dis_layers)
cdis_model.model_name = dis_model_name

# compile model
cdis_model.compile(loss=ls, optimizer=dis_opti, metrics=met)
# cdis_model.trainable = False
cdis_model.summary()

CGAN Discriminator Definition
Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ImgLabelsIn (InputLayer)        [(None, 16)]         0                                            
__________________________________________________________________________________________________
ImgLblEmb_1 (Embedding)         (None, 16, 142)      2130        ImgLabelsIn[0][0]                
__________________________________________________________________________________________________
ImgLblFlat (Flatten)            (None, 2272)         0           ImgLblEmb_1[0][0]                
__________________________________________________________________________________________________
ImgLblDense_1 (Dense)           (None, 7500)         17047500    ImgLblFlat[0][0]                 
______________________________________________________________

In [251]:
gen_layer_cfg = (
    lstm_units,
    gen_ldrop,
    gen_filters,
    ksize_s,
    stsize_s,
    # ksize,
    # stsize,
    in_gen_actf,
    hid_ly_actf,
    psize,
    pad,
    gen_midn,
    out_gen_act,
    rs,
    gen_reshape,
)

print(gen_layer_cfg)

(500, 0.2, 128, (4, 4), (2, 2), <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x000001E46C77F9D0>, <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x000001E46C77F070>, (2, 2), 'same', 7500, 'tanh', True, (50, 50, 3))


In [252]:
cgen_model = create_cgenerator_v2(disin_shape, genin_shape, n_labels, gen_layer_cfg)

In [253]:
# defining model
print("CGAN Generator definition")
cgen_model.model_name = gen_model_name

# NOT compile model
# gen_model.compile(loss=ls, optimizer=gan_opti, metrics=met)
cgen_model.summary()

CGAN Generator definition
Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
TxtLabelsIn (InputLayer)        [(None, 16)]         0                                            
__________________________________________________________________________________________________
TxtLblEmb_1 (Embedding)         (None, 16, 142)      2130        TxtLabelsIn[0][0]                
__________________________________________________________________________________________________
TxtLblFlat (Flatten)            (None, 2272)         0           TxtLblEmb_1[0][0]                
__________________________________________________________________________________________________
LblDense_1 (Dense)              (None, 7500)         17047500    TxtLblFlat[0][0]                 
__________________________________________________________________

In [254]:
def create_cgan(cgen_model, cdis_model, gan_cfg):

	# make weights in the discriminator not trainable
	cdis_model.trainable = False
	# get noise and label inputs from generator model
	gen_noise, gen_labels = cgen_model.input
	# get image output from the generator model
	gen_output = cgen_model.output
	# connect image output and label input from generator as inputs to discriminator
	gan_output = cdis_model([gen_output, gen_labels])
	# define gan model as taking noise and label and outputting a classification
	cgan_model = Model([gen_noise, gen_labels], gan_output)
	# compile model
	cgan_model.compile(loss=gan_cfg[0], optimizer=gan_cfg[1], metrics=gan_cfg[2])
	# cgan_model.compile(loss=gan_cfg[0], optimizer=gan_cfg[1])#, metrics=gan_cfg[2])
	return cgan_model

In [255]:
cgan_layer_cfg = (
    ls,
    gan_opti,
    met,
)

In [256]:
print("CGAN Model definition")
gan_model = create_cgan(cgen_model, cdis_model, cgan_layer_cfg)
gan_model.summary()

CGAN Model definition
Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
TxtLabelsIn (InputLayer)        [(None, 16)]         0                                            
__________________________________________________________________________________________________
TxtLblEmb_1 (Embedding)         (None, 16, 142)      2130        TxtLabelsIn[0][0]                
__________________________________________________________________________________________________
TxtLblFlat (Flatten)            (None, 2272)         0           TxtLblEmb_1[0][0]                
__________________________________________________________________________________________________
LblDense_1 (Dense)              (None, 7500)         17047500    TxtLblFlat[0][0]                 
______________________________________________________________________

In [257]:
# saving model topology into png files
export_model(cgen_model, model_fn_path, gen_model_name, timestamp)
export_model(cdis_model, model_fn_path, dis_model_name, timestamp)
export_model(gan_model, model_fn_path, gan_model_name, timestamp)

In [258]:
# config for training
fn_config = (model_fn_path, report_fn_path, dis_model_name, gen_model_name, gan_model_name)
check_epochs = 10

In [259]:
# dividing according to train/test proportions
# Xt_train, Xt_test, Xi_train, Xi_test = train_test_split(X_txt, X_img, train_size = train_split, test_size = test_split, random_state = randseed)
# Xi_train, Xi_test, y_train, y_test = train_test_split(X_img, y, train_size = train_split, test_size = test_split, random_state = randseed)

In [260]:
print(X_img.shape, X_txt.shape, y.shape, y_labels.shape)

(59, 400, 400, 3) (59, 15, 142) (59, 1) (59, 16)


In [261]:
train(cgen_model, cdis_model, gan_model, X_img, X_txt, y, y_labels, epo, bs, check_epochs, fn_config)
# train_good(gen_model, dis_model, gan_model, X_img, X_txt, y, epo, bs, check_epochs, fn_config)

>>> Epoch: 1, B/Ep: 1/1, Batch S: 32 -> [R-Dis loss: 1.014, acc: 0.250] || [F-Dis loss: 4.067, acc: 0.062] || [Gen loss: 0.602, acc: 0.000]
Epoch: 1 Saving the training progress...
No handles with labels found to put in legend.
Batch Size 16 -> Samples: Fake: 16 & Real: 16
>>> Test Fake -> Acc: 0.062 || Loss: 0.694
>>> Test Real -> Acc: 0.938 || Loss: 0.663
Epoch:  1 elapsed time: 36.36 [s]
>>> Epoch: 2, B/Ep: 1/1, Batch S: 32 -> [R-Dis loss: 0.493, acc: 0.500] || [F-Dis loss: 1.063, acc: 0.062] || [Gen loss: 0.687, acc: 0.000]
Epoch:  2 elapsed time: 25.03 [s]
>>> Epoch: 3, B/Ep: 1/1, Batch S: 32 -> [R-Dis loss: 0.787, acc: 0.250] || [F-Dis loss: 0.842, acc: 0.062] || [Gen loss: 0.675, acc: 0.000]
Epoch:  3 elapsed time: 25.49 [s]
>>> Epoch: 4, B/Ep: 1/1, Batch S: 32 -> [R-Dis loss: 0.588, acc: 0.688] || [F-Dis loss: 0.736, acc: 0.062] || [Gen loss: 0.648, acc: 0.000]
Epoch:  4 elapsed time: 27.96 [s]
>>> Epoch: 5, B/Ep: 1/1, Batch S: 32 -> [R-Dis loss: 0.424, acc: 0.812] || [F-Dis lo